In [2]:
import os
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
pwd

In [3]:
cd /content/drive/MyDrive/GomokuAI

/content/drive/MyDrive/GomokuAI


# Git

In [ ]:
!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), 387 bytes | 0 bytes/s, done.
From https://github.com/onetwothr1/GomokuAI
   6f6a2e0..ea27938  main       -> origin/main
Updating 6f6a2e0..ea27938
Fast-forward
 utils.py | 8 ++++++++
 1 file changed, 8 insertions(+)


In [ ]:
!git add .

In [ ]:
!git config --global user.email kimjh1101@snu.ac.kr 
!git config --global user.name onetwotr1

In [ ]:
!git commit -m "multiple changes"

In [ ]:
!git push origin main

# Train

In [24]:
import importlib
import train, utils
importlib.reload(train)
importlib.reload(utils)

<module 'utils' from '/content/drive/MyDrive/GomokuAI/utils.py'>

alphazero 0 -> 750번 자체 대국 12시간

alphazero 750 -> 500번 자체 대국 8시간

alphazero 1250 -> 500번 자체 대국 20시간 (rollout 400)

alphazero 1750

In [ ]:
!python train.py -m "models/alphazero 1250.pt" -save "models/l2 0.001" -d "experience/alphazero 1250 self-play 500x8.pickle" -n 1000 -lr 0.001 --lr-decay 0.95 --early-stop 10

alphazero 0 80.pt를 선택! -> alphazero 750
750 vs 0 => 9:1

c=1.0 vs c=2.0 => 12:2 (p-value 0.012)

c=1.0 vs c=1.5 => 10:0

weight2 epoch 109 낙찰 -> alphazero 1250.pt


alphazero 1250 vs alphazero 750 => 20 : 0

alphazero 1250
rollout 400 vs 100 => 10:0

non weighted loss, no l2 constraint. epoch 60 선택 -> alpahzero 1750

왠진 모르겠는데 l2 사용한 게 조금 더 빠르게 loss 떨어짐. policy, value 모두. 다음엔 ㅣ2사용할것

1750 : 1250 => 10:0

# 성능 비교

In [26]:
import importlib
import compare_performance, bot_v_bot, human_v_bot, utils
importlib.reload(compare_performance)
importlib.reload(bot_v_bot)
importlib.reload(human_v_bot)
importlib.reload(utils)

<module 'utils' from '/content/drive/MyDrive/GomokuAI/utils.py'>

In [ ]:
import importlib
import agent.alphazero_agent
importlib.reload(agent.alphazero_agent)

<module 'agent.alphazero_agent' from '/content/drive/MyDrive/GomokuAI/agent/alphazero_agent.py'>

In [ ]:
!python bot_v_bot.py

In [ ]:
!python compare_performance.py -m1 "models/alphazero 1750.pt" -m2 "models/alphazero 1250 103.pt" -n 10 --verbose 4 --use-model-name

Which one is black? Type 1 or 2.
1: ●, 2: ○
1
Stone color is now set.

  0% 0/10 [00:00<?, ?it/s]

////////  1th Game  ////////
Black: alphazero 1750, White: alphazero 1250 103
----------------------------
 8                   
 7                   
 6                   
 5                   
 4                   
 3                   
 2                   
 1                   
 0                   
   A B C D E F G H J
average depth: 2.06, max depth: 4
A4     visit 127  prior 0.020  value -0.06
D7     visit   9  prior 0.025  value -0.02
A3     visit   8  prior 0.022  value 0.05
E0     visit   7  prior 0.021  value -0.13
F1     visit   7  prior 0.025  value -0.07
D3     visit   7  prior 0.015  value -0.03
H3     visit   7  prior 0.025  value 0.01
B5     visit   7  prior 0.025  value -0.02
E5     visit   7  prior 0.014  value -0.04
A7     visit   7  prior 0.007  value -0.26
----------------------------
Player.black (alphazero 1750) A4
 8                   
 7                   
 6     

# self play

In [ ]:
import importlib
import bot_v_bot, utils, agent.alphazero_agent, compare_performance, self_play
# import net.alphazero_net
# importlib.reload(bot_v_bot)
# importlib.reload(compare_performance)
importlib.reload(utils)
importlib.reload(self_play)
# importlib.reload(agent.alphazero_agent)
# importlib.reload(net.alphazero_net)

<module 'self_play' from '/content/drive/MyDrive/GomokuAI/self_play.py'>

In [ ]:
for i in range(1):
  !python self_play.py -n 1 -m "models/alphazero 1250.pt" --file-num {i} -rollout 400 -c 2 --noise-intensity 0.25 --alpha 0.1 --reward-decay 0.92 --verbose 3

# 데이터 합치기

In [ ]:
import os
_list = os.listdir('experience')

In [ ]:
from pprint import pprint
files = [i for i in _list if '1250' in i and '.pickle' in i]
print(len(files))
pprint(files)

In [ ]:
import importlib
import experience
importlib.reload(experience)

<module 'experience' from '/content/drive/MyDrive/GomokuAI/experience.py'>

In [ ]:
from experience import *

combine_saved_experiences(['experience/' + f for f in files], "experience/alphazero 1250 self-play 500.pickle")

In [ ]:
from experience import *

combined = ExperienceCollector()
combined.load_experience("experience/alphazero 1250 self-play 500.pickle")
augmented = augmentation(combined, 9)
print(len(augmented))
print(len(augmented.mcts_probs))
print(len(augmented.rewards))

91808
91808
91808


In [ ]:
augmented.save_experience("experience/alphazero 1250 self-play 500*8.pickle")